In [2]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone -q
from google.colab import userdata
import os


=== 컨텍스트 ===
- 메뉴명: L..A갈비구이, 칼로리: 310.8


In [ ]:
os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


In [4]:
import os
from typing import List, Dict
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

# 1. Pinecone 인스턴스 & 인덱스
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV     = "us-east-1"
INDEX_NAME       = "food-index"
EMBED_MODEL      = "text-embedding-3-small"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)

# 벡터스토어 준비
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
)

# 2. 메뉴 이름을 받아 Pinecone에서 검색
def search_menu(menu_name: str, k: int = 1) -> List[Dict]:
    """메뉴명을 임베딩해 Pinecone에서 유사 레시피 k개 반환"""
    return vector_store.similarity_search_with_score(
        query=menu_name,
        k=k
    )

# 3. 검색 결과를 컨텍스트 문자열로 변환
def build_context(matches: List[Dict]) -> str:
    """
    Pinecone 검색 결과(matches)의 metadata에서
    RCP_NM(메뉴명)과 INFO_ENG(칼로리)를 추출해 컨텍스트 문자열 반환
    """
    lines = []
    for doc, score in matches:
        meta = doc.metadata or {}
        name = meta.get("RCP_NM", "알 수 없는 메뉴")
        kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
        lines.append(f"- 메뉴명: {name}, 칼로리: {kcal}")
    return "\n".join(lines)

# 4. 전체 파이프라인 함수
def get_menu_context(menu_name: str) -> str:
    """
    BLIP가 예측한 메뉴명을 입력받아
    Pinecone 검색 → 컨텍스트 문자열 반환
    """
    matches = search_menu(menu_name, k=1)
    return build_context(matches)

# 5. test
if __name__ == "__main__":
    blip_predicted_name = "파전"  # BLIP 결과 예시
    context = get_menu_context(blip_predicted_name)
    print("=== 컨텍스트 ===")
    print(context)

=== 컨텍스트 ===
- 메뉴명: 퓨전떡갈비, 칼로리: 195.85


In [25]:
import os
from typing import List, Dict, Tuple
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import openai  # LLM 호출용

# 1. Pinecone & 인덱스 초기화
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME       = "food-index"
EMBED_MODEL      = "text-embedding-3-small"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# 2. Pinecone 검색
def search_menu(menu_name: str, k: int = 1) -> List[Tuple]:
    return vector_store.similarity_search_with_score(query=menu_name, k=k)

# 3. Pinecone 결과 → 컨텍스트 변환
def build_context(matches: List[Tuple]) -> str:
    lines = []
    for doc, score in matches:
        meta = doc.metadata or {}
        name = meta.get("RCP_NM", "알 수 없는 메뉴")
        kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
        lines.append(f"- 메뉴명: {name}, 칼로리: {kcal} (유사도: {score:.2f})")
    return "\n".join(lines)

# 4. LLM에게 칼로리만 물어보기
def ask_llm_calorie(menu_name: str) -> str:
    prompt = (
        f"다음 음식의 대표적인 1인분 칼로리(kcal) 숫자만 알려주세요 ** 반드시 숫자만 반환!! **:\n"
        f"‘{menu_name}’"
    )
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0
    )
    # LLM이 "450kcal" 또는 "450" 같은 형태로 답한다고 가정
    return resp.choices[0].message.content.strip()

# 5. 최종 파이프라인 (threshold = 0.7)
def get_menu_context_with_threshold(
    menu_name: str,
    k: int = 1,
    threshold: float = 0.4
) -> str:
    matches = search_menu(menu_name, k)
    # ① 결과 없거나, ② 최고 유사도 < 임계치 → LLM fallback
    if not matches or matches[0][1] < threshold:
        calorie = ask_llm_calorie(menu_name)
        # 원본 포맷 그대로
        return f"- 메뉴명: {menu_name}, 칼로리: {calorie}"
    # 정상 검색 결과
    return build_context(matches)

# 6. 테스트
if __name__ == "__main__":
    blip_predicted_name = "햄버거"
    context = get_menu_context_with_threshold(blip_predicted_name)
    print("=== 컨텍스트 ===")
    print(context)

=== 컨텍스트 ===
- 메뉴명: 햄버거스테이크, 칼로리: 420.4 (유사도: 0.44)
